# Run the import block only

In [2]:
from dowhy import CausalModel
import pandas as pd
import numpy as np
import dowhy
from tqdm import tqdm

In [3]:
data = pd.read_csv('data/ml-100k/u.data', sep="\t", header=None)
data.columns = ['user_id', 'movie_id', 'rating', "time_stamp"]
data = data.drop('time_stamp', axis=1)

In [4]:
data["treatment"] = np.ones(data.shape[0], dtype=int)

In [5]:
data.shape

In [6]:
data.head()

,user_id,movie_id,rating,treatment
0,196,242,3,1
1,186,302,3,1
2,22,377,1,1
3,244,51,2,1
4,166,346,1,1


In [7]:
unique_users = data.user_id.unique().tolist()

In [8]:
unique_movie_ids = set(data.movie_id.unique().tolist())

In [9]:
grouped_df = data.groupby(by="user_id")

In [10]:
vertical_df = pd.DataFrame(columns=data.columns)

In [11]:
del data

In [12]:
for user in tqdm(unique_users):
    temp_1 = grouped_df.get_group(user)
    
    missing_movies = list(unique_movie_ids - set(temp_1.movie_id))
    count = len(missing_movies)
    
    temp_2 = {
        "user_id": [user] * count,
        "movie_id": missing_movies,
        "rating": [0] * count,
        "treatment": [0] * count
    }

    new_rec = pd.concat([temp_1, pd.DataFrame(temp_2)], ignore_index=True)
    vertical_df = pd.concat([vertical_df, new_rec], ignore_index=True)
    
    del new_rec

100%|██████████| 943/943 [00:38<00:00, 24.32it/s] 


In [13]:
vertical_df.shape

In [14]:
# run it only when you want to create a csv of preprocessed data for dowhy causal model

#vertical_df.to_csv("data/preprocess_data/preproc_100k_dowhy.csv", index=False)

# Execute the code from here

In [5]:
vertical_df = pd.read_csv("data/preprocess_data/preproc_100k_dowhy.csv")

In [6]:
vertical_df.treatment = vertical_df.treatment.astype('bool')
vertical_df.user_id = vertical_df.user_id.astype(np.uint16)
vertical_df.movie_id = vertical_df.movie_id.astype(np.uint16)
vertical_df.rating = vertical_df.rating.astype(np.uint8)

In [7]:
vertical_df.head()

,user_id,movie_id,rating,treatment
0,196,242,3,True
1,196,393,4,True
2,196,381,4,True
3,196,251,3,True
4,196,655,5,True


In [8]:
train = vertical_df.sample(frac = 0.8) 

In [9]:
test = vertical_df.drop(train.index)

In [10]:
model=CausalModel(
        data = train,
        treatment='treatment',
        outcome='rating',
        common_causes = ["user_id", "movie_id"]
        )

INFO:dowhy.causal_graph:If this is observed data (not from a randomized experiment), there might always be missing confounders. Adding a node named "Unobserved Confounders" to reflect this.
INFO:dowhy.causal_model:Model to find the causal effect of treatment ['treatment'] on outcome ['rating']


In [17]:
identified_estimand = model.identify_effect()
print(identified_estimand)

INFO:dowhy.causal_identifier:Common causes of treatment and outcome:['user_id', 'movie_id', 'U']


WARN: Do you want to continue by ignoring any unobserved confounders? (use proceed_when_unidentifiable=True to disable this prompt) [y/n] n


INFO:dowhy.causal_identifier:Instrumental variables for treatment and outcome:[]


Estimand type: nonparametric-ate
### Estimand : 1
Estimand name: backdoor
Estimand expression:
     d                                            
────────────(Expectation(rating|user_id,movie_id))
d[treatment]                                      
Estimand assumption 1, Unconfoundedness: If U→{treatment} and U→rating then P(rating|treatment,user_id,movie_id,U) = P(rating|treatment,user_id,movie_id)
### Estimand : 2
Estimand name: iv
No such variable found!



In [18]:
estimate = model.estimate_effect(identified_estimand, 
                                 method_name="backdoor.linear_regression", 
                                 test_significance=True)

INFO:dowhy.causal_estimator:INFO: Using Linear Regression Estimator
INFO:dowhy.causal_estimator:b: rating~treatment+user_id+movie_id


MemoryError: Unable to allocate 24.8 GiB for an array with shape (1268901, 2624) and data type object

In [ ]:
print(estimate)

In [ ]:
print("Causal Estimate is " + str(estimate.value))

In [ ]:
refute_results=model.refute_estimate(identified_estimand, estimate,
        method_name="random_common_cause")

In [ ]:
print(refute_results)